In [1]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import load_model
import os


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

2023-05-24 07:41:03.788656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 07:41:03.919546: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-24 07:41:04.283836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/themandalorian/anaconda3/envs/ml-on-mc/lib/
2023-05-24 07:41:04.283886: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: lib

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']
    
    
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

# create data generator
datagen = ImageDataGenerator(zoom_range=[0.9,1.1],
                             rotation_range=30,
                             brightness_range=[0.9,1.1],
                             width_shift_range=0.3, 
                             height_shift_range=0.3,
                             vertical_flip=True,
                             horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000


In [3]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += '\nstatic const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'static const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data) :

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

In [24]:
model_10 = Sequential()
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# compile model
opt = SGD(lr=0.001, momentum=0.9)

model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_tiny_Base-CubeAI_5Conv.h5', save_freq='epoch')

model_10.summary()

# Train and evaluate the quantization aware model
model_10.fit(x_train,y_train, batch_size=64,epochs=1000,callbacks=[checkpoint_callback],
             validation_data=(x_test, y_test))

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_123 (Conv2D)         (None, 32, 32, 8)         224       
                                                                 
 conv2d_124 (Conv2D)         (None, 32, 32, 16)        1168      
                                                                 
 max_pooling2d_74 (MaxPoolin  (None, 16, 16, 16)       0         
 g2D)                                                            
                                                                 
 dropout_95 (Dropout)        (None, 16, 16, 16)        0         
                                                                 
 conv2d_125 (Conv2D)         (None, 16, 16, 32)        4640      
                                                                 
 max_pooling2d_75 (MaxPoolin  (None, 8, 8, 32)         0         
 g2D)                                                

Epoch 37/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.1508 - accuracy: 0.5978 - val_loss: 1.0465 - val_accuracy: 0.6446
Epoch 38/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.1573 - accuracy: 0.5943 - val_loss: 1.0205 - val_accuracy: 0.6423
Epoch 39/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.1441 - accuracy: 0.6001 - val_loss: 0.9885 - val_accuracy: 0.6599
Epoch 40/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.1415 - accuracy: 0.6032 - val_loss: 0.9224 - val_accuracy: 0.6834
Epoch 41/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.1340 - accuracy: 0.6040 - val_loss: 0.9451 - val_accuracy: 0.6714
Epoch 42/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.1394 - accuracy: 0.6025 - val_loss: 0.9737 - val_accuracy: 0.6672
Epoch 43/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.1347 - accuracy: 0.6044 - val_loss: 0

Epoch 93/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.0216 - accuracy: 0.6485 - val_loss: 0.8053 - val_accuracy: 0.7155
Epoch 94/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.0166 - accuracy: 0.6501 - val_loss: 0.7941 - val_accuracy: 0.7269
Epoch 95/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.0116 - accuracy: 0.6498 - val_loss: 0.7818 - val_accuracy: 0.7294
Epoch 96/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.0064 - accuracy: 0.6552 - val_loss: 0.8723 - val_accuracy: 0.7026
Epoch 97/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.0058 - accuracy: 0.6553 - val_loss: 0.8763 - val_accuracy: 0.6971
Epoch 98/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.0024 - accuracy: 0.6558 - val_loss: 0.7617 - val_accuracy: 0.7382
Epoch 99/1000
782/782 [==============================] - 17s 22ms/step - loss: 1.0054 - accuracy: 0.6526 - val_loss: 0

In [25]:
# load the pre-saved model
model_10 = load_model('model_10_tiny_Base-CubeAI_5Conv.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

In [32]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_tiny_Base-CubeAI_5Conv_PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_tiny_Base-CubeAI_5Conv_PTQ is {}%".format(accuracy_score*100))

== Input details ==
name: serving_default_conv2d_123_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of model_10_tiny_Base-CubeAI_5Conv_PTQ is 74.68%


In [28]:
c_model_name = 'cifar10_tiny_base_cubeAI_5conv_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [17]:
model_10 = Sequential()
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# compile model
opt = SGD(lr=0.001, momentum=0.9)

model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_Base-CubeAI_Tiniest.h5', save_freq='epoch')

model_10.summary()

# Train and evaluate the quantization aware model
model_10.fit(x_train,y_train, batch_size=64,epochs=1000,callbacks=[checkpoint_callback],
             validation_data=(x_test, y_test))

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_52 (Conv2D)          (None, 32, 32, 8)         224       
                                                                 
 conv2d_53 (Conv2D)          (None, 32, 32, 8)         584       
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 16, 16, 8)        0         
 g2D)                                                            
                                                                 
 dropout_49 (Dropout)        (None, 16, 16, 8)         0         
                                                                 
 conv2d_54 (Conv2D)          (None, 16, 16, 16)        1168      
                                                                 
 conv2d_55 (Conv2D)          (None, 16, 16, 16)        2320      
                                                     

Epoch 39/1000
782/782 [==============================] - 4s 4ms/step - loss: 1.4531 - accuracy: 0.4632 - val_loss: 1.3301 - val_accuracy: 0.5100
Epoch 40/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.4436 - accuracy: 0.4652 - val_loss: 1.3016 - val_accuracy: 0.5204
Epoch 41/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.4367 - accuracy: 0.4700 - val_loss: 1.2844 - val_accuracy: 0.5272
Epoch 42/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.4376 - accuracy: 0.4715 - val_loss: 1.2788 - val_accuracy: 0.5333
Epoch 43/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.4265 - accuracy: 0.4727 - val_loss: 1.2728 - val_accuracy: 0.5300
Epoch 44/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.4218 - accuracy: 0.4764 - val_loss: 1.2993 - val_accuracy: 0.5253
Epoch 45/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.4208 - accuracy: 0.4793 - val_loss: 1.2657 - val_ac

782/782 [==============================] - 3s 4ms/step - loss: 1.2471 - accuracy: 0.5550 - val_loss: 1.0914 - val_accuracy: 0.6042
Epoch 96/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.2454 - accuracy: 0.5557 - val_loss: 1.0800 - val_accuracy: 0.6042
Epoch 97/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.2456 - accuracy: 0.5548 - val_loss: 1.0638 - val_accuracy: 0.6158
Epoch 98/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.2418 - accuracy: 0.5579 - val_loss: 1.0937 - val_accuracy: 0.6054
Epoch 99/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.2376 - accuracy: 0.5602 - val_loss: 1.0897 - val_accuracy: 0.6085
Epoch 100/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.2359 - accuracy: 0.5629 - val_loss: 1.0720 - val_accuracy: 0.6139
Epoch 101/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.2346 - accuracy: 0.5605 - val_loss: 1.0697 - val_accuracy: 0.61

782/782 [==============================] - 3s 4ms/step - loss: 1.1542 - accuracy: 0.5880 - val_loss: 1.0606 - val_accuracy: 0.6175
Epoch 152/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.1522 - accuracy: 0.5912 - val_loss: 0.9865 - val_accuracy: 0.6420
Epoch 153/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.1556 - accuracy: 0.5929 - val_loss: 0.9940 - val_accuracy: 0.6377
Epoch 154/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.1544 - accuracy: 0.5918 - val_loss: 0.9809 - val_accuracy: 0.6405
Epoch 155/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.1515 - accuracy: 0.5932 - val_loss: 1.0069 - val_accuracy: 0.6319
Epoch 156/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.1474 - accuracy: 0.5957 - val_loss: 1.0532 - val_accuracy: 0.6169
Epoch 157/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.1495 - accuracy: 0.5949 - val_loss: 0.9744 - val_accuracy: 

782/782 [==============================] - 3s 4ms/step - loss: 1.0986 - accuracy: 0.6133 - val_loss: 0.9512 - val_accuracy: 0.6553
Epoch 208/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.0989 - accuracy: 0.6132 - val_loss: 0.9448 - val_accuracy: 0.6563
Epoch 209/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.0913 - accuracy: 0.6144 - val_loss: 0.9496 - val_accuracy: 0.6544
Epoch 210/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.0910 - accuracy: 0.6138 - val_loss: 0.9226 - val_accuracy: 0.6630
Epoch 211/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.0906 - accuracy: 0.6146 - val_loss: 1.0051 - val_accuracy: 0.6348
Epoch 212/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.0903 - accuracy: 0.6162 - val_loss: 0.9145 - val_accuracy: 0.6703
Epoch 213/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.0880 - accuracy: 0.6153 - val_loss: 0.9247 - val_accuracy: 

...

782/782 [==============================] - 3s 4ms/step - loss: 0.9502 - accuracy: 0.6712 - val_loss: 0.7799 - val_accuracy: 0.7216
Epoch 768/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9453 - accuracy: 0.6734 - val_loss: 0.7950 - val_accuracy: 0.7181
Epoch 769/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9435 - accuracy: 0.6707 - val_loss: 0.8393 - val_accuracy: 0.7051
Epoch 770/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9451 - accuracy: 0.6703 - val_loss: 0.8187 - val_accuracy: 0.7121
Epoch 771/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9436 - accuracy: 0.6716 - val_loss: 0.7780 - val_accuracy: 0.7219
Epoch 772/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9386 - accuracy: 0.6750 - val_loss: 0.8133 - val_accuracy: 0.7082
Epoch 773/1000
782/782 [==============================] - 4s 4ms/step - loss: 0.9473 - accuracy: 0.6713 - val_loss: 0.7825 - val_accuracy: 

782/782 [==============================] - 3s 4ms/step - loss: 0.9446 - accuracy: 0.6718 - val_loss: 0.7877 - val_accuracy: 0.7198
Epoch 824/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9484 - accuracy: 0.6715 - val_loss: 0.7905 - val_accuracy: 0.7167
Epoch 825/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9465 - accuracy: 0.6705 - val_loss: 0.8187 - val_accuracy: 0.7113
Epoch 826/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9419 - accuracy: 0.6735 - val_loss: 0.7887 - val_accuracy: 0.7167
Epoch 827/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9436 - accuracy: 0.6735 - val_loss: 0.7908 - val_accuracy: 0.7172
Epoch 828/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9437 - accuracy: 0.6723 - val_loss: 0.8029 - val_accuracy: 0.7157
Epoch 829/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9466 - accuracy: 0.6721 - val_loss: 0.8210 - val_accuracy: 

782/782 [==============================] - 3s 4ms/step - loss: 0.9364 - accuracy: 0.6749 - val_loss: 0.8102 - val_accuracy: 0.7114
Epoch 880/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9354 - accuracy: 0.6760 - val_loss: 0.8207 - val_accuracy: 0.7085
Epoch 881/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9403 - accuracy: 0.6743 - val_loss: 0.8147 - val_accuracy: 0.7097
Epoch 882/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9417 - accuracy: 0.6732 - val_loss: 0.7846 - val_accuracy: 0.7221
Epoch 883/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9408 - accuracy: 0.6737 - val_loss: 0.7848 - val_accuracy: 0.7197
Epoch 884/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9447 - accuracy: 0.6729 - val_loss: 0.7924 - val_accuracy: 0.7161
Epoch 885/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9340 - accuracy: 0.6783 - val_loss: 0.8099 - val_accuracy: 

782/782 [==============================] - 3s 4ms/step - loss: 0.9419 - accuracy: 0.6732 - val_loss: 0.7871 - val_accuracy: 0.7202
Epoch 936/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9324 - accuracy: 0.6759 - val_loss: 0.7791 - val_accuracy: 0.7236
Epoch 937/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9367 - accuracy: 0.6757 - val_loss: 0.7722 - val_accuracy: 0.7248
Epoch 938/1000
782/782 [==============================] - 4s 4ms/step - loss: 0.9350 - accuracy: 0.6751 - val_loss: 0.7923 - val_accuracy: 0.7187
Epoch 939/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9338 - accuracy: 0.6782 - val_loss: 0.8102 - val_accuracy: 0.7141
Epoch 940/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9322 - accuracy: 0.6759 - val_loss: 0.7971 - val_accuracy: 0.7163
Epoch 941/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9316 - accuracy: 0.6770 - val_loss: 0.7765 - val_accuracy: 

782/782 [==============================] - 3s 4ms/step - loss: 0.9369 - accuracy: 0.6749 - val_loss: 0.7880 - val_accuracy: 0.7202
Epoch 992/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9352 - accuracy: 0.6748 - val_loss: 0.7826 - val_accuracy: 0.7254
Epoch 993/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9367 - accuracy: 0.6780 - val_loss: 0.7952 - val_accuracy: 0.7187
Epoch 994/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9315 - accuracy: 0.6745 - val_loss: 0.7812 - val_accuracy: 0.7230
Epoch 995/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9357 - accuracy: 0.6788 - val_loss: 0.7757 - val_accuracy: 0.7255
Epoch 996/1000
782/782 [==============================] - 4s 4ms/step - loss: 0.9316 - accuracy: 0.6760 - val_loss: 0.7960 - val_accuracy: 0.7191
Epoch 997/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9310 - accuracy: 0.6749 - val_loss: 0.7860 - val_accuracy: 

In [18]:
# load the pre-saved model
model_10 = load_model('model_10_Base-CubeAI_Tiniest.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.7946 - accuracy: 0.7166
Test accuracy: 0.7166000008583069


In [19]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_Base-CubeAI_Tiniest_PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_Base-CubeAI_Tiniest_PTQ is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmpto8l95qw/assets


INFO:tensorflow:Assets written to: /tmp/tmpto8l95qw/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-23 23:51:38.978026: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-23 23:51:38.978051: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-23 23:51:38.978707: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpto8l95qw
2023-05-23 23:51:38.983153: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-23 23:51:38.983171: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpto8l95qw
2023-05-23 23:51:38.994650: I tensorflow/

== Input details ==
name: serving_default_conv2d_52_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of model_10_Base-CubeAI_Tiniest_PTQ is 71.8%


In [20]:
c_model_name = 'cifar10_base_cubeAI_tiniest_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [21]:
model_10 = Sequential()
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_tiny_Base-CubeAI_5Conv_QAT.h5', save_freq='epoch')

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=1000,
                      callbacks=[checkpoint_callback],validation_data=(x_test, y_test))

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 32, 32, 3)        3         
 er)                                                             
                                                                 
 quant_conv2d_58 (QuantizeWr  (None, 32, 32, 8)        243       
 apperV2)                                                        
                                                                 
 quant_conv2d_59 (QuantizeWr  (None, 32, 32, 16)       1203      
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_39 (Qua  (None, 16, 16, 16)       1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_dropout_53 (QuantizeW  (None, 16, 16, 16)     

Epoch 30/1000
782/782 [==============================] - 9s 12ms/step - loss: 1.5304 - accuracy: 0.4346 - val_loss: 1.6212 - val_accuracy: 0.4161
Epoch 31/1000
782/782 [==============================] - 9s 12ms/step - loss: 1.5179 - accuracy: 0.4394 - val_loss: 1.7452 - val_accuracy: 0.3786
Epoch 32/1000
782/782 [==============================] - 9s 11ms/step - loss: 1.5122 - accuracy: 0.4399 - val_loss: 1.6276 - val_accuracy: 0.4199
Epoch 33/1000
782/782 [==============================] - 9s 11ms/step - loss: 1.4994 - accuracy: 0.4482 - val_loss: 1.6645 - val_accuracy: 0.4095
Epoch 34/1000
782/782 [==============================] - 9s 11ms/step - loss: 1.4884 - accuracy: 0.4511 - val_loss: 1.6205 - val_accuracy: 0.4229
Epoch 35/1000
782/782 [==============================] - 9s 11ms/step - loss: 1.4749 - accuracy: 0.4568 - val_loss: 1.5663 - val_accuracy: 0.4477
Epoch 36/1000
782/782 [==============================] - 9s 12ms/step - loss: 1.4647 - accuracy: 0.4631 - val_loss: 1.6609 -

782/782 [==============================] - 9s 11ms/step - loss: 1.1602 - accuracy: 0.5851 - val_loss: 1.1437 - val_accuracy: 0.5960
Epoch 87/1000
782/782 [==============================] - 9s 11ms/step - loss: 1.1636 - accuracy: 0.5831 - val_loss: 1.1957 - val_accuracy: 0.5798
Epoch 88/1000
782/782 [==============================] - 9s 12ms/step - loss: 1.1542 - accuracy: 0.5878 - val_loss: 1.1128 - val_accuracy: 0.6186
Epoch 89/1000
782/782 [==============================] - 9s 12ms/step - loss: 1.1552 - accuracy: 0.5866 - val_loss: 1.1152 - val_accuracy: 0.6077
Epoch 90/1000
782/782 [==============================] - 9s 12ms/step - loss: 1.1492 - accuracy: 0.5878 - val_loss: 1.0979 - val_accuracy: 0.6132
Epoch 91/1000
782/782 [==============================] - 9s 12ms/step - loss: 1.1432 - accuracy: 0.5913 - val_loss: 1.1323 - val_accuracy: 0.6012
Epoch 92/1000
782/782 [==============================] - 9s 12ms/step - loss: 1.1465 - accuracy: 0.5900 - val_loss: 1.0946 - val_accuracy:

Epoch 142/1000
782/782 [==============================] - 7s 9ms/step - loss: 1.0221 - accuracy: 0.6374 - val_loss: 0.9266 - val_accuracy: 0.6755
Epoch 143/1000
782/782 [==============================] - 9s 11ms/step - loss: 1.0252 - accuracy: 0.6348 - val_loss: 0.9074 - val_accuracy: 0.6822
Epoch 144/1000
782/782 [==============================] - 8s 10ms/step - loss: 1.0191 - accuracy: 0.6415 - val_loss: 0.9213 - val_accuracy: 0.6771
Epoch 145/1000
782/782 [==============================] - 8s 10ms/step - loss: 1.0164 - accuracy: 0.6405 - val_loss: 1.0270 - val_accuracy: 0.6446
Epoch 146/1000
782/782 [==============================] - 8s 10ms/step - loss: 1.0183 - accuracy: 0.6383 - val_loss: 0.9358 - val_accuracy: 0.6707
Epoch 147/1000
782/782 [==============================] - 8s 10ms/step - loss: 1.0118 - accuracy: 0.6416 - val_loss: 0.9506 - val_accuracy: 0.6681
Epoch 148/1000
782/782 [==============================] - 8s 10ms/step - loss: 1.0108 - accuracy: 0.6428 - val_loss: 0.

Epoch 198/1000
782/782 [==============================] - 9s 11ms/step - loss: 0.9235 - accuracy: 0.6743 - val_loss: 0.8686 - val_accuracy: 0.6907
Epoch 199/1000
782/782 [==============================] - 8s 11ms/step - loss: 0.9224 - accuracy: 0.6784 - val_loss: 0.8131 - val_accuracy: 0.7163
Epoch 200/1000
782/782 [==============================] - 9s 11ms/step - loss: 0.9240 - accuracy: 0.6747 - val_loss: 0.8516 - val_accuracy: 0.7043
Epoch 201/1000
782/782 [==============================] - 9s 11ms/step - loss: 0.9282 - accuracy: 0.6743 - val_loss: 0.8269 - val_accuracy: 0.7055
Epoch 202/1000
782/782 [==============================] - 9s 11ms/step - loss: 0.9193 - accuracy: 0.6742 - val_loss: 0.8066 - val_accuracy: 0.7143
Epoch 203/1000
782/782 [==============================] - 9s 11ms/step - loss: 0.9220 - accuracy: 0.6765 - val_loss: 0.8500 - val_accuracy: 0.6996
Epoch 204/1000
782/782 [==============================] - 9s 11ms/step - loss: 0.9163 - accuracy: 0.6746 - val_loss: 0

In [4]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_tiny_Base-CubeAI_5Conv_QAT.tflite", "wb").write(tflite_model_quant_int8_qat)

tflite_interpreter = tf.lite.Interpreter("model_10_tiny_Base-CubeAI_5Conv_QAT.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_tiny_Base-CubeAI_5Conv_QAT is {}%".format(accuracy_score*100))

== Input details ==
name: serving_default_conv2d_58_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.uint8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.uint8'>
Accuracy of model_10_tiny_Base-CubeAI_5Conv_QAT is 80.75%


In [23]:
c_model_name = 'cifar10_tiny_base_cubeAI_5conv_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))

In [24]:
model_10 = Sequential()
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_Base-CubeAI_Tiniest_QAT.h5', save_freq='epoch')

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=1000,
                      callbacks=[checkpoint_callback],validation_data=(x_test, y_test))

In [5]:
# evaluate the model on the test set
#quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
#print('Quantization aware training loss: ', quant_loss)
#print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
#def representative_data_gen():
#    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
#        yield [input_value]

#converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
#converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8

#tflite_model_quant_int8_qat = converter.convert()

#open("model_10_Base-CubeAI_Tiniest_QAT.tflite", "wb").write(tflite_model_quant_int8_qat)

tflite_interpreter = tf.lite.Interpreter("model_10_Base-CubeAI_Tiniest_QAT.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_Base-CubeAI_Tiniest_QAT is {}%".format(accuracy_score*100))

== Input details ==
name: serving_default_conv2d_63_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.uint8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.uint8'>
Accuracy of model_10_Base-CubeAI_Tiniest_QAT is 72.22%


In [26]:
c_model_name = 'cifar10_base_cubeAI_tiniest_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))